In [27]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random
import os
import ast

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [28]:
#replace as needed
folder_name = "Supplementary Materials"
followers_path = os.path.join(folder_name, "followers_US.txt")
completed_path = os.path.join(folder_name, "completed_accounts")

followers_US = []
# with open(r'''X\followers_US.txt''', 'r') as f: # PC
with open(followers_path,'r') as f: #List of accounts from raw followers list
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            if follower not in followers_US:
                # print(follower)
                followers_US.append(follower)
f.close()

with open(completed_path, 'r') as f: #Get list of accounts that are completed
    for line in f:
        if line.startswith('total'):
            completed_list = ast.literal_eval(line[8:])

#Filter out the ones that are completed
followers_US_set = set(followers_US)
followers_US = [f for f in followers_US_set if f not in completed_list]

print(followers_US)

print(f"Number of followers to process: {len(followers_US)}")

['ProudSocialist', 'ComicDaveSmith', 'LPTN1776', 'checkmatestate', 'ajc', 'RBReich', 'theintercept', 'TheRickWilson', 'FreemansMind96', 'AmpedUpPodcast', 'nbcnews', 'NPR', 'DailyCaller', 'Morehouse', 'LPNH', 'TheAtlantic', 'people4liberty', 'TeamWarnock', 'OversightDems', 'Weaponization', 'MeidasTouch', 'StJude', 'BadFaithPod', 'whitecoatwaste', 'InequalityMedia', 'GOP', 'Josh_Moon', 'HoustonChron', 'HannahDCox', 'KyleKulinski', 'EricTrump', 'LaraLeaTrump', 'seanhannity', 'RacePowerPolicy', 'ProjectLincoln', 'UALawSchool', 'PeoplesParty_US', 'nytimes', 'funder', 'WashingtonPost', 'GOPoversight', 'yhn', 'rbreich', 'Newsweek', 'glennkirschner2', 'DonaldJTrumpJr', 'scotthortonshow', 'PeterGHager']
Number of followers to process: 48


In [29]:
tweets_dict_US = {}

# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

In [30]:
driver = webdriver.Chrome()
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


The chromedriver version (128.0.6613.137) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (129.0.6668.70); currently, chromedriver 129.0.6668.70 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


In [31]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("rigautam@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("RichaGotem")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Sus1234!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [32]:
count = 1

keyword1 = "(Gaza* OR Israel* OR (West OR AND OR Bank) OR (Tel OR AND OR Aviv) OR (Tel OR AND OR Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"
until = "2024-09-22"
since = "2023-10-07"

unprocessed = []

for profile in tqdm(followers_US):
    if count%5 == 0:
        print("5 Accounts passed, sleeping for 7 minutes.")
        time.sleep(420)
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                try:
                    print(f'{profile} - {count} - Retry button appeared.')
                    unprocessed.append(profile)
                    for i in range(5):
                        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                        search_button.click()
                    time.sleep(100) 
                except:
                    print(f"{profile} - Failed to click Retry")

            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
            try:
                # Select the home timeline div
                search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                # Just get all the HTML, will parse later
                tweets_listed = search_results.get_attribute('innerHTML')
                time.sleep(3)

                # Append to dict
                tweets_dict_US[profile].append(tweets_listed)

            except:
                pass

    count+=1

print("Unprocessed accounts for this run:")
for profile in unprocessed:
    print(f"@{profile}\n")


  0%|          | 0/48 [00:00<?, ?it/s]

ProudSocialist


  2%|▏         | 1/48 [00:51<40:13, 51.34s/it]

ComicDaveSmith


  4%|▍         | 2/48 [02:06<49:59, 65.21s/it]

...

yhn


 88%|████████▊ | 42/48 [1:41:42<14:07, 141.23s/it]

rbreich


 88%|████████▊ | 42/48 [1:42:23<14:37, 146.28s/it]


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.70)
Stacktrace:
0   chromedriver                        0x0000000109e4f338 chromedriver + 5096248
1   chromedriver                        0x0000000109e46b6a chromedriver + 5061482
2   chromedriver                        0x00000001099d1fd0 chromedriver + 389072
3   chromedriver                        0x00000001099ba84a chromedriver + 292938
4   chromedriver                        0x00000001099ba56a chromedriver + 292202
5   chromedriver                        0x00000001099b8888 chromedriver + 284808
6   chromedriver                        0x00000001099b8dba chromedriver + 286138
7   chromedriver                        0x00000001099b7981 chromedriver + 280961
8   chromedriver                        0x00000001099dcbca chromedriver + 433098
9   chromedriver                        0x0000000109a

In [33]:
driver.quit()

In [34]:
tweets_dict_US.keys()

dict_keys(['ProudSocialist', 'ComicDaveSmith', 'LPTN1776', 'checkmatestate', 'ajc', 'RBReich', 'theintercept', 'TheRickWilson', 'FreemansMind96', 'AmpedUpPodcast', 'nbcnews', 'NPR', 'DailyCaller', 'Morehouse', 'LPNH', 'TheAtlantic', 'people4liberty', 'TeamWarnock', 'OversightDems', 'Weaponization', 'MeidasTouch', 'StJude', 'BadFaithPod', 'whitecoatwaste', 'InequalityMedia', 'GOP', 'Josh_Moon', 'HoustonChron', 'HannahDCox', 'KyleKulinski', 'EricTrump', 'LaraLeaTrump', 'seanhannity', 'RacePowerPolicy', 'ProjectLincoln', 'UALawSchool', 'PeoplesParty_US', 'nytimes', 'funder', 'WashingtonPost', 'GOPoversight', 'yhn', 'rbreich'])

In [35]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['ProudSocialist', 'ComicDaveSmith', 'LPTN1776', 'checkmatestate', 'ajc', 'RBReich', 'theintercept', 'TheRickWilson', 'FreemansMind96', 'AmpedUpPodcast', 'nbcnews', 'NPR', 'DailyCaller', 'Morehouse', 'LPNH', 'TheAtlantic', 'people4liberty', 'TeamWarnock', 'OversightDems', 'Weaponization', 'MeidasTouch', 'StJude', 'BadFaithPod', 'whitecoatwaste', 'InequalityMedia', 'GOP', 'Josh_Moon', 'HoustonChron', 'HannahDCox', 'KyleKulinski', 'EricTrump', 'LaraLeaTrump', 'seanhannity', 'RacePowerPolicy', 'ProjectLincoln', 'UALawSchool', 'PeoplesParty_US', 'nytimes', 'funder', 'WashingtonPost', 'GOPoversight', 'yhn', 'rbreich'])

In [36]:
raw_data_folder = "Raw Data"
raw_json = os.path.join(raw_data_folder, f"US_tweets_{until}_{since}.json")
cleaned_json = os.path.join(raw_data_folder, f"US_tweets_cleaned_{until}_{since}.json")

In [37]:
with open(raw_json, 'w') as file:
    json.dump(tweets_dict_US, file)

In [38]:
with open(cleaned_json, 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)